In [1]:
import warnings
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

Using TensorFlow backend.


# Reading Datasets

In [2]:
# Main Dataset
data= pd.read_csv("Data/NN5_interpolated.csv",header=None)
# Expert Knowledge
predictions = pd.read_csv("Data/NN5_theta_25_horg.csv",index_col=0,skiprows = [1])
print(data.shape)
print(predictions.shape)

(111, 791)
(111, 594)


# Functions to Make Inputs

In [3]:
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
#     depth=data.shape[2]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size+1-horizon):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def make_k_input(data,window_size,horizon):
    length = data.shape[0]
    output= np.zeros([length-window_size+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
#     print(str(extra))
    data = np.append(data,np.zeros([horizon-extra]))
#     print(data)
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_k_input(data,window_size,horizon):
    length = data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    
#     data = np.append(data,np.zeros([horizon-extra]))
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)    

# Evaluation Function (SMAPE)

In [4]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200.0 * np.mean(diff)

# Auto-Encoder for Expert Knowledge

In [7]:
theta_length = predictions.shape[0]
horizon=56
window_size=70
theta_all=np.zeros(0)
for i in range(theta_length):
    current_pred= np.asarray(predictions.iloc[i].dropna().values,dtype=float)
    theta_all=np.concatenate((theta_all,current_pred),axis=0)
theta_input = make_k_input(theta_all,window_size,horizon)
normalized_theta_input=np.zeros(theta_input.shape)
for j in range(theta_input.shape[0]):
    scaler = MinMaxScaler()
    scaler.fit(theta_input[j])
    normalized_theta_input[j]= scaler.transform(theta_input[j])
#######################################################################################
norm_reshaped_input=normalized_theta_input.reshape(normalized_theta_input.shape[0],56)
ncol = normalized_theta_input.shape[1] #56

encoding_dim = 32
input_dim = Input(shape = (ncol, ))
# Encoder Layers
encoded1 = Dense(48, activation = 'relu')(input_dim)
encoded2 = Dense(40, activation = 'relu')(encoded1)
encoded3 = Dense(encoding_dim, activation = 'relu')(encoded2)
# Decoder Layers
decoded1 = Dense(40, activation = 'relu')(encoded3)
decoded2 = Dense(48, activation = 'sigmoid')(decoded1)
decoded3 = Dense(ncol, activation = 'sigmoid')(decoded2)
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded3)
autoencoder.compile(optimizer = 'adam', loss = 'mse')

autoencoder.fit(norm_reshaped_input, norm_reshaped_input, epochs = 500, batch_size = 32, shuffle = False,verbose=2)
encoder = Model(inputs = input_dim, outputs = encoded3)
encoded_input = Input(shape = (encoding_dim, ))
auto_out = np.array(encoder.predict(norm_reshaped_input))

new_theta_input=auto_out.reshape(auto_out.shape[0],auto_out.shape[1],1)
autoencoder.save('Output/autoencoder_model_NN5_w70.h5')
encoder.save('Output/encoder_model_NN5_w70.h5')
np.savetxt('Output/new_theta_input_NN5_w70.csv',auto_out, fmt='%1.3f',delimiter=',')    


Epoch 1/500
 - 7s - loss: 0.0264
Epoch 2/500
 - 5s - loss: 0.0121
Epoch 3/500
 - 6s - loss: 0.0082
Epoch 4/500
 - 6s - loss: 0.0057
Epoch 5/500
 - 7s - loss: 0.0051
Epoch 6/500
 - 6s - loss: 0.0047
Epoch 7/500
 - 7s - loss: 0.0042
Epoch 8/500
 - 7s - loss: 0.0040
Epoch 9/500
 - 6s - loss: 0.0039
Epoch 10/500
 - 5s - loss: 0.0038
Epoch 11/500
 - 6s - loss: 0.0036
Epoch 12/500
 - 5s - loss: 0.0035
Epoch 13/500
 - 5s - loss: 0.0034
Epoch 14/500
 - 5s - loss: 0.0034
Epoch 15/500
 - 5s - loss: 0.0033
Epoch 16/500
 - 5s - loss: 0.0032
Epoch 17/500
 - 5s - loss: 0.0030
Epoch 18/500
 - 5s - loss: 0.0029
Epoch 19/500
 - 5s - loss: 0.0028
Epoch 20/500
 - 5s - loss: 0.0027
Epoch 21/500
 - 5s - loss: 0.0026
Epoch 22/500
 - 5s - loss: 0.0026
Epoch 23/500
 - 5s - loss: 0.0026
Epoch 24/500
 - 5s - loss: 0.0025
Epoch 25/500
 - 5s - loss: 0.0025
Epoch 26/500
 - 5s - loss: 0.0024
Epoch 27/500
 - 5s - loss: 0.0023
Epoch 28/500
 - 5s - loss: 0.0023
Epoch 29/500
 - 5s - loss: 0.0023
Epoch 30/500
 - 5s - lo

Epoch 472/500
 - 7s - loss: 0.0013
Epoch 473/500
 - 9s - loss: 0.0013
Epoch 474/500
 - 7s - loss: 0.0013
Epoch 475/500
 - 6s - loss: 0.0013
Epoch 476/500
 - 6s - loss: 0.0013
Epoch 477/500
 - 5s - loss: 0.0013
Epoch 478/500
 - 6s - loss: 0.0013
Epoch 479/500
 - 5s - loss: 0.0013
Epoch 480/500
 - 6s - loss: 0.0013
Epoch 481/500
 - 6s - loss: 0.0013
Epoch 482/500
 - 5s - loss: 0.0013
Epoch 483/500
 - 5s - loss: 0.0013
Epoch 484/500
 - 6s - loss: 0.0013
Epoch 485/500
 - 6s - loss: 0.0013
Epoch 486/500
 - 6s - loss: 0.0013
Epoch 487/500
 - 5s - loss: 0.0013
Epoch 488/500
 - 5s - loss: 0.0013
Epoch 489/500
 - 5s - loss: 0.0013
Epoch 490/500
 - 5s - loss: 0.0013
Epoch 491/500
 - 5s - loss: 0.0013
Epoch 492/500
 - 5s - loss: 0.0013
Epoch 493/500
 - 5s - loss: 0.0013
Epoch 494/500
 - 6s - loss: 0.0013
Epoch 495/500
 - 6s - loss: 0.0013
Epoch 496/500
 - 6s - loss: 0.0013
Epoch 497/500
 - 6s - loss: 0.0013
Epoch 498/500
 - 5s - loss: 0.0013
Epoch 499/500
 - 5s - loss: 0.0013
Epoch 500/500
 - 5s 

# Case 1 : Only Data (75%)

In [11]:
warnings.filterwarnings('ignore')
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    final_smape=np.zeros(data_length)
    for y in range(data_length):                
        num_test=56
        horizon=56
        window_size=70
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] 
        series_data = series_d[:-num_test]
        series_length = series_data.size
        current_test=series_d[-num_test:]
        n_val = int(np.round(series_length*.2))
        
        train = series_data[:-n_val]
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)
        
        scaler = MinMaxScaler()
        for j in range(train_sequence[0].shape[0]):
            scaler.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
        for j in range(val_sequence[0].shape[0]):
            scaler.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])
        for j in range(test_sequence[0].shape[0]):
            scaler.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
            
        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             

        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
################       Neural Network Configuration         ########################
####################################################################################        
        tf.compat.v1.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        
        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.00001))(branch_3)
        
        model=Model(inputs=[input_data],outputs=net)
        callback = ModelCheckpoint(filepath='NetworkWeights_Checkpoints/checkpoint_NN5_01_%i.h5' %y,monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001))
        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=12,shuffle=True, epochs=200,verbose=0)
        
        model.load_weights('NetworkWeights_Checkpoints/checkpoint_NN5_01_%i.h5' %y)
        pred=model.predict({'input_data':test_input})
        pred=scaler.inverse_transform(pred)

        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test)       
        final_smape[y]=AAA
        pbar.update(1)

np.savetxt('Output/prediction_NN5_01.csv',final_predictions, fmt='%1.3f',delimiter=',')
model.summary()
print('-----------------')
print("SMAPE:")
print (sum(final_smape)/data_length)    

100%|██████████████████████████████████████████████████████████████████████████████| 111/111 [2:13:20<00:00, 74.29s/it]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 70, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 70, 32)            128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 70, 64)            6208      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 64)            12352     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4480)              0         
_________________________________________________________________
dense_final (Dense)          (None, 56)                250936    
Total params: 269,624
Trainable params: 269,624
Non-trainable params: 0
_________________________________________________________________
----

# Only data (100%)

In [6]:
warnings.filterwarnings('ignore')
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    final_smape=np.zeros(data_length)
    for y in range(data_length):                
        num_test=56
        horizon=56
        window_size=70
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        #rr = current_row.size
        #rr = int(np.floor(rr*.25))
        #series_d=current_row[rr:] 
        series_d=current_row
        series_data = series_d[:-num_test]
        series_length = series_data.size
        current_test=series_d[-num_test:]
        n_val = int(np.round(series_length*.2))
        
        train = series_data[:-n_val]
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)
        
        scaler = MinMaxScaler()
        for j in range(train_sequence[0].shape[0]):
            scaler.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
        for j in range(val_sequence[0].shape[0]):
            scaler.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])
        for j in range(test_sequence[0].shape[0]):
            scaler.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
            
        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             

        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
################       Neural Network Configuration         ########################
####################################################################################        
        tf.compat.v1.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        
        branch_0 = Conv1D(128,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(128,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(128,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.00001))(branch_3)
        
        model=Model(inputs=[input_data],outputs=net)
        callback = ModelCheckpoint(filepath='NetworkWeights_Checkpoints/checkpoint_NN5_011_%i.h5' %y,monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001))
        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=12,shuffle=True, epochs=200,verbose=0)
        
        model.load_weights('NetworkWeights_Checkpoints/checkpoint_NN5_011_%i.h5' %y)
        pred=model.predict({'input_data':test_input})
        pred=scaler.inverse_transform(pred)

        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test)       
        final_smape[y]=AAA
        pbar.update(1)

np.savetxt('Output/prediction_NN5_011.csv',final_predictions, fmt='%1.3f',delimiter=',')
model.summary()
print('-----------------')
print("SMAPE:")
print (sum(final_smape)/data_length)    

100%|█████████████████████████████████████████████████████████████████████████████| 111/111 [5:40:12<00:00, 212.15s/it]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 70, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 70, 128)           512       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 70, 128)           49280     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 128)           49280     
_________________________________________________________________
flatten_1 (Flatten)          (None, 8960)              0         
_________________________________________________________________
dense_final (Dense)          (None, 56)                501816    
Total params: 600,888
Trainable params: 600,888
Non-trainable params: 0
_________________________________________________________________
----

# Case 3: Data + Expert Knowledge

In [27]:
warnings.filterwarnings('ignore')
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    final_smape=np.zeros(data_length)    
    for y in range(data_length):                    
        num_test=56
        horizon=56
        window_size=70
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] 
        series_data = series_d[:-num_test]
        series_length = series_data.size
        current_test=series_d[-num_test:]
        n_val = int(np.round(series_length*.2))
        
        train = series_data[:-n_val]
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)
        
        scaler = MinMaxScaler()
        for j in range(train_sequence[0].shape[0]):
            scaler.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
        for j in range(val_sequence[0].shape[0]):
            scaler.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])
        for j in range(test_sequence[0].shape[0]):
            scaler.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])

        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             
        
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        train_p = series_pred[:-n_val]                                        
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
                
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        scaler_pred = MinMaxScaler()
        for j in range(train_pred.shape[0]):
            scaler_pred.fit(train_pred[j])
            train_pred_norm[j]= scaler_pred.transform(train_pred[j])
        for j in range(val_pred.shape[0]):            
            scaler_pred.fit(val_pred[j])
            val_pred_norm[j]= scaler_pred.transform(val_pred[j])
        for j in range(test_pred.shape[0]):            
            scaler_pred.fit(test_pred[j])
            test_pred_norm[j]= scaler_pred.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])
##########################################################################################################################
        ncol = 56
        encoding_dim = 32
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('Output/encoder_model_NN5_w70.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))        
        
        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                                    
####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.compat.v1.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,encoding_dim),name='input_pred')

        encoded_0=Reshape((1,32))(input_pred)
        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded_0],axis=1)
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.00001))(branch_3)
        
        model=Model(inputs=[input_data,input_pred],outputs=net)
        #callback = ModelCheckpoint(filepath='NetworkWeights_Checkpoints/checkpoint_NN5_02_70_%i.h5' %y,monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001))
        #model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=12,shuffle=True, epochs=300,verbose=0)
        model.load_weights('NetworkWeights_Checkpoints/checkpoint_NN5_02_70_%i.h5' %y)
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)
        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test)
        final_smape[y]=AAA
        pbar.update(1)

np.savetxt('Output/prediction_NN5_02_70.csv',final_predictions, fmt='%1.3f',delimiter=',')
model.summary()
print('-----------------')
print("SMAPE:")
print (sum(final_smape)/data_length)

100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [01:18<00:00,  1.44it/s]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 70, 1)        0                                            
__________________________________________________________________________________________________
input_pred (InputLayer)         (None, 32)           0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 70, 32)       128         input_data[0][0]                 
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 32)        0           input_pred[0][0]                 
__________________________________________________________________________________________________
concatenat

# Case 4: Only Data 
## The Network is trained by whole Dataset

In [6]:
data_length = data.shape[0]
horizon=56
window_size=56
num_test=56
for i in range(data_length): #data_length
    current_row= np.asarray(data.iloc[i].dropna().values,dtype=float)
    rr = current_row.size
    rr = int(np.floor(rr*.25))
    series_data = current_row[:-num_test]
    series_length = series_data.size
    n_val = int(np.round(series_length*.2))  
    train = series_data[:-n_val]
    val = series_data[-(n_val+window_size):]
    train_sequence = make_input(train, window_size,horizon)
    val_sequence = make_input(val,window_size,horizon)
    
    temp_train_x=train_sequence[0]
    temp_train_x=temp_train_x.reshape(temp_train_x.shape[0],temp_train_x.shape[1])
    temp_train_y=train_sequence[1]
    
    temp_val_x=val_sequence[0]
    temp_val_x=temp_val_x.reshape(temp_val_x.shape[0],temp_val_x.shape[1])
    temp_val_y=val_sequence[1]
    if(i==0):
        data_train_x=temp_train_x
        data_train_y=temp_train_y
        data_val_x=temp_val_x
        data_val_y=temp_val_y
    else:
        data_train_x=np.concatenate((data_train_x,temp_train_x),axis=0)
        data_train_y=np.concatenate((data_train_y,temp_train_y),axis=0)
        data_val_x=np.concatenate((data_val_x,temp_val_x),axis=0)
        data_val_y=np.concatenate((data_val_y,temp_val_y),axis=0)

data_train_x=data_train_x.reshape(data_train_x.shape[0],data_train_x.shape[1],1)
data_train_y=data_train_y.reshape(data_train_y.shape[0],data_train_y.shape[1],1)
data_val_x=data_val_x.reshape(data_val_x.shape[0],data_val_x.shape[1],1)
data_val_y=data_val_y.reshape(data_val_y.shape[0],data_val_y.shape[1],1)

scaler = MinMaxScaler()
for j in range(data_train_x.shape[0]):
    scaler.fit(data_train_x[j])
    data_train_x[j]= scaler.transform(data_train_x[j])
    data_train_y[j]= scaler.transform(data_train_y[j])
for j in range(data_val_x.shape[0]):
    scaler.fit(data_val_x[j])
    data_val_x[j]= scaler.transform(data_val_x[j])
    data_val_y[j]= scaler.transform(data_val_y[j])
data_train_y=data_train_y.reshape(data_train_y.shape[0],data_train_y.shape[1])
data_val_y=data_val_y.reshape(data_val_y.shape[0],data_val_y.shape[1])
#########################################################################################
#######################################################################################
warnings.filterwarnings('ignore')
tf.compat.v1.reset_default_graph()
K.clear_session()
input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        
branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
#branch_3 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_2)
#branch_4 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_3)

branch_5=Flatten()(branch_2)

net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.00001))(branch_5)

model=Model(inputs=[input_data],outputs=net)
callback = ModelCheckpoint(filepath='temp/checkpoint_NN5_01.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001))
model.fit({'input_data':data_train_x},data_train_y,validation_data=[[data_val_x],data_val_y],callbacks=[callback],batch_size=16,shuffle=True, epochs=100,verbose=0)
model.save('Output/trained_all_NN5.h5')

warnings.filterwarnings('ignore')
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    final_smape=np.zeros(data_length)
    for y in range(data_length):                
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] 
        series_data = series_d[:-num_test]
        series_length = series_data.size
        current_test=series_d[-num_test:]
        n_val = int(np.round(series_length*.2))
               
        test = series_d[-(num_test+window_size):]
        test_sequence = nonov_make_input(test,window_size,horizon)
        test_sequence_norm = deepcopy(test_sequence)
        
        scaler = MinMaxScaler()
        
        for j in range(test_sequence[0].shape[0]):
            scaler.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
            
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             

        test_input = x_test

        
        pred=model.predict({'input_data':test_input})
        pred=scaler.inverse_transform(pred)

        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test)       
        final_smape[y]=AAA
        pbar.update(1)

np.savetxt('Output/prediction_NN5_01_wholedata.csv',final_predictions, fmt='%1.3f',delimiter=',')
model.summary()
print('-----------------')
print("SMAPE:")
print (sum(final_smape)/data_length)    

100%|███████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 253.72it/s]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 56, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 56, 32)            128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 56, 64)            6208      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 56, 64)            12352     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3584)              0         
_________________________________________________________________
dense_final (Dense)          (None, 56)                200760    
Total params: 219,448
Trainable params: 219,448
Non-trainable params: 0
_________________________________________________________________
----

# Case 5: Data + Expert Knowledge
## The Network is trained by whole Dataset

In [9]:
data_length = data.shape[0]
horizon=56
window_size=56
num_test=56
for i in range(data_length): #data_length
    current_row= np.asarray(data.iloc[i].dropna().values,dtype=float)
    rr = current_row.size
    rr = int(np.floor(rr*.25))
    series_d=current_row[rr:]
    series_data = series_d[:-num_test]
    series_length = series_data.size
    n_val = int(np.round(series_length*.2))  
    train = series_data[:-n_val]
    val = series_data[-(n_val+window_size):]
    train_sequence = make_input(train, window_size,horizon)
    val_sequence = make_input(val,window_size,horizon)
    
    temp_train_x=train_sequence[0]
    temp_train_x=temp_train_x.reshape(temp_train_x.shape[0],temp_train_x.shape[1])
    temp_train_y=train_sequence[1]
    
    temp_val_x=val_sequence[0]
    temp_val_x=temp_val_x.reshape(temp_val_x.shape[0],temp_val_x.shape[1])
    temp_val_y=val_sequence[1]
    if(i==0):
        data_train_x=temp_train_x
        data_train_y=temp_train_y
        data_val_x=temp_val_x
        data_val_y=temp_val_y
    else:
        data_train_x=np.concatenate((data_train_x,temp_train_x),axis=0)
        data_train_y=np.concatenate((data_train_y,temp_train_y),axis=0)
        data_val_x=np.concatenate((data_val_x,temp_val_x),axis=0)
        data_val_y=np.concatenate((data_val_y,temp_val_y),axis=0)
##################################
##################################
    current_pred= np.asarray(predictions.iloc[i].dropna().values,dtype=float)
    series_p=current_pred
    series_pred=series_p[:-num_test]    
    train_p = series_pred[:-n_val]                                        
    val_p = series_pred[-(n_val+window_size):]
    train_pred = make_k_input(train_p,window_size,horizon)
    val_pred = make_k_input(val_p,window_size,horizon)
    
    temp_train_p_x=train_pred
    temp_train_p_x=temp_train_p_x.reshape(temp_train_p_x.shape[0],temp_train_p_x.shape[1])
    
    temp_val_p_x=val_pred
    temp_val_p_x=temp_val_p_x.reshape(temp_val_p_x.shape[0],temp_val_p_x.shape[1])
    
    scaler = MinMaxScaler()
    for j in range(temp_train_p_x.shape[0]):
        scaler.fit(temp_train_p_x[j].reshape(temp_train_p_x[j].shape[0],1))
        temp_train_p_x[j]= scaler.transform(temp_train_p_x[j].reshape(temp_train_p_x[j].shape[0],1)).reshape(temp_train_p_x[j].shape[0])
    for j in range(temp_val_p_x.shape[0]):
        scaler.fit(temp_val_p_x[j].reshape(temp_val_p_x[j].shape[0],1))
        temp_val_p_x[j]= scaler.transform(temp_val_p_x[j].reshape(temp_val_p_x[j].shape[0],1)).reshape(temp_val_p_x[j].shape[0])
    
    ncol = 56
    encoding_dim = 32
    input_dim = Input(shape = (ncol, ))
    warnings.filterwarnings('ignore')

    input_dim = Input(shape = (ncol, ))
    encoded_input = Input(shape = (encoding_dim, ))
    encoder=load_model('Output/encoder_model_NN5_w70.h5')
    
    temp_train_p_x=np.array(encoder.predict(temp_train_p_x))
    temp_val_p_x=np.array(encoder.predict(temp_val_p_x))
    
    if(i==0):
        data_train_p_x=temp_train_p_x
        data_val_p_x=temp_val_p_x
    else:
        data_train_p_x=np.concatenate((data_train_p_x,temp_train_p_x),axis=0)
        data_val_p_x=np.concatenate((data_val_p_x,temp_val_p_x),axis=0)
##################################
##################################
data_train_x=data_train_x.reshape(data_train_x.shape[0],data_train_x.shape[1],1)
data_train_y=data_train_y.reshape(data_train_y.shape[0],data_train_y.shape[1],1)
data_val_x=data_val_x.reshape(data_val_x.shape[0],data_val_x.shape[1],1)
data_val_y=data_val_y.reshape(data_val_y.shape[0],data_val_y.shape[1],1)

scaler = MinMaxScaler()
for j in range(data_train_x.shape[0]):
    scaler.fit(data_train_x[j])
    data_train_x[j]= scaler.transform(data_train_x[j])
    data_train_y[j]= scaler.transform(data_train_y[j])
for j in range(data_val_x.shape[0]):
    scaler.fit(data_val_x[j])
    data_val_x[j]= scaler.transform(data_val_x[j])
    data_val_y[j]= scaler.transform(data_val_y[j])
data_train_y=data_train_y.reshape(data_train_y.shape[0],data_train_y.shape[1])
data_val_y=data_val_y.reshape(data_val_y.shape[0],data_val_y.shape[1])
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
#########################################################################################
#######################################################################################
warnings.filterwarnings('ignore')
tf.compat.v1.reset_default_graph()
K.clear_session()
input_data= Input(batch_shape=(None,window_size,1),name='input_data')
input_pred=Input(batch_shape=(None,32),name='input_pred')

encoded_0=Reshape((1,32))(input_pred)
branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
branch_0=concatenate([branch_0,encoded_0],axis=1)
branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
#branch_3 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_2)
#branch_4 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_3)

branch_5=Flatten()(branch_2)

net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.00001))(branch_5)

model=Model(inputs=[input_data,input_pred],outputs=net)
callback = ModelCheckpoint(filepath='temp/checkpoint_NN5_02.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001))
model.fit({'input_data':data_train_x,'input_pred':data_train_p_x},data_train_y,validation_data=[[data_val_x,data_val_p_x],data_val_y],callbacks=[callback],batch_size=16,shuffle=True, epochs=100,verbose=0)
model.save('Output/trained_all_NN5_02.h5')

warnings.filterwarnings('ignore')
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    final_smape=np.zeros(data_length)
    for y in range(data_length):                
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] 
        series_data = series_d[:-num_test]
        series_length = series_data.size
        current_test=series_d[-num_test:]
        n_val = int(np.round(series_length*.2))
               
        test = series_d[-(num_test+window_size):]
        test_sequence = nonov_make_input(test,window_size,horizon)
        test_sequence_norm = deepcopy(test_sequence)
        
        scaler = MinMaxScaler()
        
        for j in range(test_sequence[0].shape[0]):
            scaler.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
            
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             

        test_input = x_test
                        
        
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        test_p = series_p[-(num_test+window_size):]
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
                
        test_pred_norm=np.zeros(test_pred.shape)
    
        scaler_pred = MinMaxScaler()
        for j in range(test_pred.shape[0]):            
            scaler_pred.fit(test_pred[j])
            test_pred_norm[j]= scaler_pred.transform(test_pred[j])
        
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])
##########################################################################################################################
        ncol = 56
        encoding_dim = 32
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('Output/encoder_model_NN5_w70.h5')
        test_pred_auto=np.array(encoder.predict(test_pred_norm))        
        test_pred_auto=pd.DataFrame(test_pred_auto)    

        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)

        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test)       
        final_smape[y]=AAA
        pbar.update(1)

np.savetxt('Output/prediction_NN5_02_wholedata.csv',final_predictions, fmt='%1.3f',delimiter=',')
model.summary()
print('-----------------')
print("SMAPE:")
print (sum(final_smape)/data_length)    

W1114 11:18:20.634135 11100 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [01:05<00:00,  1.02s/it]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 56, 1)        0                                            
__________________________________________________________________________________________________
input_pred (InputLayer)         (None, 32)           0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 56, 32)       128         input_data[0][0]                 
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 32)        0           input_pred[0][0]                 
__________________________________________________________________________________________________
concatenat